In [ ]:
from tdmclient import ClientAsync
import vision.ComputerVision
import time
import cv2
import motion_functions
import matplotlib.pyplot as plt
import filtering
import numpy as np

R = 0.021 # [m] The radius of the Thymio's wheels
d = 0.095 # [m] The wheelbase of the Thymio
dt = 0.05 # [s] Time delta between steps

vis = vision.ComputerVision.Vision()

client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

last_image_time = time.time()
dt = 0.05
path = []

state = 0 # FSM state
target_node = 0 # Initial node
goal_reached = False
kidnapped = 0
obstacle_detected = 0
rotation_done = 1

while True:
    while state == 0: # Goal, robot and path aquisition state
        print(state)
        while time.time()-last_image_time < dt: # Aquire a new image every dt seconds
            continue
        vis.show()
        vis.update()
        last_image_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        path = vis.shortest_path
        if len(path)>=1:
            state = 1
            rob_pos = [vis.robot.x, vis.robot.y, vis.robot.angle]

    while state == 1: # Drive towards goal state
        print(state)
        while time.time()-last_image_time < dt: # Aquire a new image every dt seconds
            continue
        vis.show()
        vis.update()
        
        await client.sleep(0.01)
        await node.wait_for_variables()
        
        # Get proximity sensors' values
        kidnapped, obstacle_detected = motion_functions.get_sensors(node, rotation_done)
        # Checking for unforseen obstacle
        if obstacle_detected:
            state = 0
            obstacle_detected = 0
            vis.shortest_path = [] # Deleting the previous path to make sure a new one is computed
            await motion_functions.stop(node)
            break
        
        # Checking for kidnapping
        if kidnapped:
            state = 3 # Kidnapped state
            vis.shortest_path = [] # Deleting the previous path to make sure a new one is computed
            await motion_functions.stop(node)
            break
        
        # Setting up and running the Kalman Filter
        speed_left = node.v.motor.left.speed
        speed_right = node.v.motor.right.speed
        prev_angle = rob_pos[2]
        #rob_pos = filtering.run_filter(speed_right, speed_left, prev_angle, vis,R,d,time.time()-last_image_time)
        rob_pos = [vis.robot.x, vis.robot.y, vis.robot.angle]
        # Checking if we arrived at the next node
        arrived_node = motion_functions.close_coordinates(rob_pos[0], rob_pos[1], path[target_node][0], path[target_node][1])
        if (arrived_node):
            if (target_node) < len(path):
                target_node += 1
            else:
                print("ARRIVED")
                await motion_functions.stop(node)
                goal_reached = True
                break # Exiting
        
        # Rotating and driving towards the next node
        angle = motion_functions.compute_movement([rob_pos[0], rob_pos[1]], [path[target_node][0], path[target_node][1]], rob_pos[2])
        print("measured angle" , rob_pos[2], "computed_angle, ", angle)
        if abs(angle) < rob_pos[2] - 0.1 or abs(angle) > rob_pos[2] + 0.1:
            await motion_functions.rotate(angle, node)
            rotation_done = 1
        else:
            await motion_functions.drive(node)
            
        last_image_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    while state == 3: # State in case of kidnapping
        print(state)
        while time.time()-last_image_time < dt: # Aquire a new image every dt seconds
            continue
        vis.show()
        vis.update()
        
        kidnapped, obstacle_detected = motion_functions.get_sesnors(node, rotation_done)
        if kidnapped == 0:
            state == 0
        
        last_image_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    if goal_reached:
        break

del vis
await node.unlock()